**Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler

In [3]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [4]:
df1 = pd.read_csv("train.csv")
df1.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


**EDA**

In [5]:
#Shape of the dataset
df1.shape

(58645, 13)

In [6]:
#Checking for null values
df1.isnull().sum()

id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

In [7]:
#Descriptive Analysis of Numerical Variables
df1.describe()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,58645.000000,5.864500e+04,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000
mean,29322.000000,27.550857,6.404617e+04,4.701015,9217.556518,10.677874,0.159238,5.813556,0.142382
std,16929.497605,6.033216,3.793111e+04,3.959784,5563.807384,3.034697,0.091692,4.029196,0.349445
min,0.000000,20.000000,4.200000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,14661.000000,23.000000,4.200000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,29322.000000,26.000000,5.800000e+04,4.000000,8000.000000,10.750000,0.140000,4.000000,0.000000
75%,43983.000000,30.000000,7.560000e+04,7.000000,12000.000000,12.990000,0.210000,8.000000,0.000000
max,58644.000000,123.000000,1.900000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [8]:
#Dropping 'id' column
df1.drop(columns='id', axis=1, inplace=True)

In [9]:
#Checking if the target column - loan_status is balanced
df1['loan_status'].value_counts()

loan_status
0    50295
1     8350
Name: count, dtype: int64

**Handling Categorical Columns**

Person_Home_Ownership

In [10]:
df1['person_home_ownership'].value_counts()

person_home_ownership
RENT        30594
MORTGAGE    24824
OWN          3138
OTHER          89
Name: count, dtype: int64

In [11]:
#One Hot Encode the column
df1 = pd.get_dummies(df1, columns=['person_home_ownership'], prefix='ownership')

# Display the updated dataframe
df1.head()

,person_age,person_income,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT
0,37,35000,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0,False,False,False,True
1,22,56000,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0,False,False,True,False
2,29,28800,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0,False,False,True,False
3,30,70000,14.0,VENTURE,B,12000,11.11,0.17,N,5,0,False,False,False,True
4,22,60000,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0,False,False,False,True


Loan_Intent

In [12]:
df1['loan_intent'].value_counts()

loan_intent
EDUCATION            12271
MEDICAL              10934
PERSONAL             10016
VENTURE              10011
DEBTCONSOLIDATION     9133
HOMEIMPROVEMENT       6280
Name: count, dtype: int64

In [13]:
#One Hot Encode the column
df1 = pd.get_dummies(df1, columns=['loan_intent'], prefix='intent')

# Display the updated dataframe
df1.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE
0,37,35000,0.0,B,6000,11.49,0.17,N,14,0,False,False,False,True,False,True,False,False,False,False
1,22,56000,6.0,C,4000,13.35,0.07,N,2,0,False,False,True,False,False,False,False,True,False,False
2,29,28800,8.0,A,6000,8.90,0.21,N,10,0,False,False,True,False,False,False,False,False,True,False
3,30,70000,14.0,B,12000,11.11,0.17,N,5,0,False,False,False,True,False,False,False,False,False,True
4,22,60000,2.0,A,6000,6.92,0.10,N,3,0,False,False,False,True,False,False,False,True,False,False


In [14]:
#New shape after OHE
df1.shape

(58645, 20)

Loan_Grade

In [15]:
df1['loan_grade'].value_counts()

loan_grade
A    20984
B    20400
C    11036
D     5034
E     1009
F      149
G       33
Name: count, dtype: int64

In [16]:
# Define custom mapping where A has the highest label
loan_grade_mapping = {'A': 6, 'B': 5, 'C': 4, 'D': 3, 'E': 2, 'F': 1, 'G': 0}

# Map the loan_grade column using the custom mapping
df1['loan_grade_encoded'] = df1['loan_grade'].map(loan_grade_mapping)

In [17]:
df1.drop(columns='loan_grade', axis=1, inplace=True)

In [18]:
df1.sample(n=5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE,loan_grade_encoded
31140,22,49600,5.0,8400,7.14,0.17,N,3,0,False,False,False,True,False,False,False,False,True,False,6
16223,26,50000,9.0,3000,5.79,0.06,N,2,0,True,False,False,False,True,False,False,False,False,False,6
47258,22,80000,5.0,7000,13.99,0.09,Y,4,0,False,False,False,True,True,False,False,False,False,False,4
8248,26,110000,10.0,12000,10.74,0.11,N,2,0,True,False,False,False,True,False,False,False,False,False,5
31809,43,75000,0.0,10000,10.36,0.13,N,16,0,False,False,False,True,False,False,True,False,False,False,5


cb_person_default_on_file

In [19]:
df1['cb_person_default_on_file'].value_counts()

cb_person_default_on_file
N    49943
Y     8702
Name: count, dtype: int64

In [20]:
# Label Encode the value
le = LabelEncoder()

# Fit and transform
df1['cb_person_default_on_file'] = le.fit_transform(df1['cb_person_default_on_file'])

In [21]:
# To check the mapping of the original labels to the encoded values
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)

{'N': 0, 'Y': 1}


In [22]:
df1.sample(n=5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE,loan_grade_encoded
52006,27,60000,11.0,4500,9.25,0.07,0,10,0,True,False,False,False,False,False,False,True,False,False,5
10015,36,33000,2.0,7500,13.92,0.22,1,14,0,False,False,False,True,False,False,False,False,False,True,4
28902,27,55000,4.0,15000,16.00,0.27,0,7,0,True,False,False,False,False,False,False,False,False,True,2
49499,28,84996,12.0,8000,13.61,0.09,1,10,0,True,False,False,False,False,False,False,False,True,False,4
9403,46,65000,2.0,10000,13.49,0.15,1,13,0,True,False,False,False,False,False,False,True,False,False,4


**Test Data**

In [23]:
df_test = pd.read_csv("test (1).csv")
df_test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [24]:
#Handling 'id' column
id = df_test['id']
df_test.drop(columns='id', axis=1, inplace=True)
df_test.sample(n=5)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
10613,28,46500,MORTGAGE,13.0,PERSONAL,A,6000,7.49,0.13,N,6
10710,29,120000,MORTGAGE,3.0,HOMEIMPROVEMENT,A,20000,6.17,0.17,N,6
20994,25,78000,MORTGAGE,9.0,MEDICAL,B,14200,10.99,0.18,N,3
15738,36,57996,RENT,3.0,DEBTCONSOLIDATION,C,7000,12.84,0.12,Y,15
23355,23,40000,RENT,1.0,EDUCATION,A,5000,7.74,0.13,N,3


In [25]:
#One Hot Encode the column - person_home_ownership and loan_intent
df_test = pd.get_dummies(df_test, columns=['person_home_ownership', 'loan_intent'], prefix=['ownership','intent'])

# Display the updated dataframe
df_test.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE
0,23,69000,3.0,F,25000,15.76,0.36,N,2,False,False,False,True,False,False,True,False,False,False
1,26,96000,6.0,C,10000,12.68,0.10,Y,4,True,False,False,False,False,False,False,False,True,False
2,26,30000,5.0,E,4000,17.19,0.13,Y,2,False,False,False,True,False,False,False,False,False,True
3,33,50000,4.0,A,7000,8.90,0.14,N,7,False,False,False,True,True,False,False,False,False,False
4,26,102000,8.0,D,15000,16.32,0.15,Y,4,True,False,False,False,False,False,True,False,False,False


In [26]:
# Map the loan_grade column using the custom mapping
df_test['loan_grade_encoded'] = df_test['loan_grade'].map(loan_grade_mapping)
df_test.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE,loan_grade_encoded
0,23,69000,3.0,F,25000,15.76,0.36,N,2,False,False,False,True,False,False,True,False,False,False,1
1,26,96000,6.0,C,10000,12.68,0.10,Y,4,True,False,False,False,False,False,False,False,True,False,4
2,26,30000,5.0,E,4000,17.19,0.13,Y,2,False,False,False,True,False,False,False,False,False,True,2
3,33,50000,4.0,A,7000,8.90,0.14,N,7,False,False,False,True,True,False,False,False,False,False,6
4,26,102000,8.0,D,15000,16.32,0.15,Y,4,True,False,False,False,False,False,True,False,False,False,3


In [27]:
#Drop the loan_grade column
df_test.drop(columns='loan_grade', axis=1, inplace=True)
df_test.sample(n=5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE,loan_grade_encoded
7157,28,96000,3.0,17600,10.99,0.18,N,9,False,False,False,True,False,False,True,False,False,False,5
23491,31,38400,4.0,2500,13.98,0.07,Y,6,False,False,False,True,True,False,False,False,False,False,4
38818,32,55000,3.0,4200,13.79,0.08,N,6,False,False,False,True,False,False,True,False,False,False,4
21930,25,66000,4.0,25000,11.48,0.38,N,2,True,False,False,False,False,False,True,False,False,False,5
37048,28,150000,12.0,20000,10.00,0.13,N,8,True,False,False,False,True,False,False,False,False,False,5


In [28]:
#Label Encode cb_person_default_on_file
# Fit and transform
df_test['cb_person_default_on_file'] = le.fit_transform(df_test['cb_person_default_on_file'])

df_test.sample(n=5)

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,ownership_MORTGAGE,ownership_OTHER,ownership_OWN,ownership_RENT,intent_DEBTCONSOLIDATION,intent_EDUCATION,intent_HOMEIMPROVEMENT,intent_MEDICAL,intent_PERSONAL,intent_VENTURE,loan_grade_encoded
15031,22,60000,6.0,12000,13.11,0.20,1,3,False,False,False,True,False,True,False,False,False,False,4
19107,33,30000,0.0,10000,7.66,0.33,0,9,False,False,False,True,False,False,False,False,False,True,6
5698,23,38400,4.0,20000,12.53,0.48,0,2,True,False,False,False,False,True,False,False,False,False,5
30767,23,70000,1.0,20000,6.54,0.29,0,2,True,False,False,False,False,False,False,False,False,True,6
12422,22,70000,3.0,25000,11.11,0.36,0,4,True,False,False,False,False,True,False,False,False,False,5


**XG Boost**

In [ ]:
"""# Convert specific object columns to category type - Only for Plain Vanilla XG Boost
categorical_columns = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
df1[categorical_columns] = df1[categorical_columns].astype('category')
df_test[categorical_columns] = df_test[categorical_columns].astype('category')"""

In [29]:
# Separate features (X) and target (y) from df1
X = df1.drop(columns=['loan_status'])
y = df1['loan_status']

In [30]:
# Split the train data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Initialize and train the model
#model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
model = 
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=42)

In [32]:
# Validate the model
y_val_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])
print(f"Validation Accuracy: {accuracy}")
print(f"Validation ROC AUC Score: {roc_auc}")

Validation Accuracy: 0.9119277005712337
Validation ROC AUC Score: 0.9410619691509687


In [33]:
# Make prediction on the unseen test data
df_test['loan_status_prediction'] = model.predict(df_test)

In [34]:
submission = pd.DataFrame({'id': id, 'loan_status': df_test['loan_status_prediction']})
submission.to_csv('submission_v5.csv', index=False)
print("Submission file created: 'submission_v5.csv'")

Submission file created: 'submission_v5.csv'
